In [1]:
import json
import os
import sys

from datasets import load_from_disk
import evaluate
import numpy as np
import torch
#from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator, EarlyStoppingCallback

sys.path.append("../transformers/src")
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator, EarlyStoppingCallback

device = "cuda:0" if torch.cuda.is_available() else "cpu"
if device == "cuda:0":    
    n_gpus = torch.cuda.device_count()
    print("GPUs:")
    for i in range(n_gpus):
        print(torch.cuda.get_device_name(i))

GPUs:
NVIDIA RTX A6000
NVIDIA GeForce RTX 2080 Ti
NVIDIA GeForce RTX 2080 Ti
NVIDIA GeForce RTX 2080 Ti
NVIDIA GeForce RTX 2080 Ti


In [2]:
# TimeSformer
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
tokenizer.pad_token = tokenizer.eos_token

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length = 50
model.config.num_beams = 4
model.config.early_stopping = True

# VideoMAE
# encoder = "MCG-NJU/videomae-base"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained(encoder)
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k600 were not used when initializing TimesformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.8.crossattention.c_proj.bias', 'h.4.crossattention.masked_bias', 'h.9.c

In [3]:
dataset = load_from_disk("dataset/processed/k600")
dataset

DatasetDict({
    train: Dataset({
        features: ['videoID', 'pixel_values', 'labels'],
        num_rows: 1016
    })
    validation: Dataset({
        features: ['videoID', 'pixel_values', 'labels'],
        num_rows: 117
    })
})

In [4]:
train_output, val_output = {}, {}
output_dir = "training/5e-5"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    fp16=True,
    predict_with_generate=True,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    dataloader_num_workers=8,
    num_train_epochs=10,
    learning_rate=5e-5,
)

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")

def metrics(eval_preds):
    preds, labels = eval_preds
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    for i, label in enumerate(labels):
        if label in val_output:
            val_output[label].append(preds[i])
        else:
            val_output[label] = [preds[i]]
            
    bleu_scores = bleu.compute(predictions=preds, references=labels, smooth=True)
    meteor_scores = meteor.compute(predictions=preds, references=labels)
    rouge_scores = rouge.compute(predictions=preds, references=labels, rouge_types=['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    result = {}
    result["bleu"] = bleu_scores["bleu"]
    result["meteor"] = meteor_scores["meteor"]
    result["rouge1"] = rouge_scores["rouge1"]
    result["rouge2"] = rouge_scores["rouge2"]
    result["rougeL"] = rouge_scores["rougeL"]
    return result

trainer = Seq2SeqTrainer(
    train_output,
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=default_data_collator,
    compute_metrics=metrics,
)

trainer.train()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/922201615/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/922201615/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/922201615/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `VisionEncoderDecoderModel.forward` and have been ignored: videoID. If videoID are not expected by `VisionEncoderDecoderModel.forward`,  you can safely ignore this message.
/home/922201615/video-caption/../transformers/src/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  w

Epoch,Training Loss,Validation Loss


Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "early_stopping": true,
  "max_length": 50,
  "num_beams": 4,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "early_stopping": true,
  "max_length": 50,
  "num_beams": 4,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "early_stopping": true,
  "max_length": 50,
  "num_beams": 4,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "early_stopping": true,
  "max_length": 50,
  "num_beams": 4,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "early_stopping": true,
  "max_length": 50,
  "num_beams": 4,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Generate config

KeyboardInterrupt: 

In [ ]:
with open(os.path.join(output_dir, "train_output.json"), "w") as file:
    file.write(json.dumps(train_output))
    
with open(os.path.join(output_dir, "val_output.json"), "w") as file:
    file.write(json.dumps(val_output))